# Unit test

In [18]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [19]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection

from skforecast.datasets import fetch_dataset



In [23]:
metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
data_test = data.loc[end_train:, :].copy()
levels = ["item_1", "item_2", "item_3"]
exog_features = 'day_of_week'

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
steps = 1
initial_train_size = 100
param_grid = {
    "alpha": np.logspace(-3, 3, 1),
}
lags_grid = [5]
param_grid = list(ParameterGrid(param_grid))

forecaster = ForecasterAutoregMultiSeries(
    regressor=Ridge(random_state=678),
    lags=5,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

forecaster = ForecasterAutoregMultiVariate(
    regressor=Ridge(random_state=678),
    lags=3,
    steps=1,
    level="item_1",
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=data.loc[:, levels],
        exog=data.loc[:, exog_features] if exog_features else None,
        initial_train_size=initial_train_size,
    )
)

results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = True
    )

results_one_step_ahead

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

,levels,lags,lags_label,params,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_scaled_error,root_mean_squared_scaled_error,alpha
0,[item_1],"[1, 2, 3, 4, 5]","[1, 2, 3, 4, 5]",{'alpha': 0.001},1.154476,2.582983,0.051834,0.774511,0.817991,0.001


In [25]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
levels = ["item_1", "item_2", "item_3"]
exog_features = ['day_of_week']

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 100
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:393: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


In [26]:
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog

series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 20
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='l1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )
]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:393: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  - additional n columns with param = value.


In [40]:
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 213
    param_grid = {
        "n_estimators": [5, 10],
        "max_depth": [2, 3]
    }
    lags_grid = [3, 5]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    # display(results_backtesting)
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )
    # display(results_one_step_ahead)

    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = LGBMRegressor(random_state=678, verbose=-1)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3, forecaster_id=1),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
        forecaster_id=2
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        forecaster_id=3
    )
]
for forecaster in forecasters:
    print(forecaster.forecaster_id)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1807: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  )


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1807: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  )


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1807: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  )


In [38]:
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)

metrics, predictions = backtesting_forecaster_multiseries(
    forecaster=forecaster,
    series=series_dict,
    exog=exog_dict,
    steps=1,
    initial_train_size=213,
    metric=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric=True,
    n_jobs='auto',
    verbose=False,
    show_progress=True
)

display(metrics)

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)
(
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_encoding,
    X_test_encoding
) = forecaster._train_test_split_one_step_ahead(
        series             = series_dict,
        exog               = exog_dict,
        initial_train_size = 213,
    )

metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
    forecaster=forecaster,
    series=series_dict,
    X_train = X_train,
    y_train= y_train,
    X_train_encoding = X_train_encoding,
    X_test = X_test,
    y_test = y_test,
    X_test_encoding = X_test_encoding,
    levels = ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
    metrics=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric = True
)
display(metrics_one_step_ahead)


pd.testing.assert_frame_equal(metrics, metrics_one_step_ahead)

TypeError: `series` must be a pandas DataFrame.

## backtesting vs _predict_and_calculate_metrics_multiseries_one_step_ahead

In [34]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [35]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False
# in backtesting_forecaster_multiseries and _predict_and_calculate_metrics_multiseries_one_step_ahead


# Data download
series = fetch_dataset(name="items_sales")
exog = pd.DataFrame(
    {
        'day_of_week': series.index.dayofweek
    },
    index = series.index
)
initial_train_size = 927

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),
    ForecasterAutoregMultiVariate(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        steps              = 1,
        level              = 'item_1',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
    )
]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series,
        exog=exog,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series,
            exog               = exog,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.794891,0.039537,0.520771


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.794891,0.039537,0.520771


In [5]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),
    ForecasterAutoregMultiVariate(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        steps              = 1,
        level              = 'item_1',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
    )
]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series_dict,
        exog=exog_dict,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series_dict,
            exog               = exog_dict,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series_dict,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:836: MissingValuesWarning: NaNs detected in `y_train`. They have been dropped because the target variable cannot have NaN values. Same rows have been dropped from `X_train` to maintain alignment. This is caused by series with interspersed NaNs. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:858: MissingValuesWarning: NaNs detected in `X_train`. Some regressors do not allow NaN values during training. If you want to drop them, set `forecaster.dropna_from_series = True`. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(


ValueError: No objects to concatenate

In [36]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecaster = ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 10,
        encoding           = 'ordinal',
        # transformer_series = StandardScaler(),
        # transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    )


metrics_backtesting, predictions = backtesting_forecaster_multiseries(
    series=series_dict,
    exog=exog_dict,
    forecaster=forecaster,
    steps=10,
    metric=metrics,
    initial_train_size = initial_train_size,
    refit=False,
    levels=['item_1', 'item_2', 'item_3'],
    add_aggregated_metric=False,
    show_progress=False
)

display(metrics_backtesting)

ValueError: No objects to concatenate